In [1]:
!pip install -q --upgrade tensorflow transformers huggingface-hub datasets evaluate
# !pip install -q evaluate
# !pip install -q --upgrade tensorflow transformers huggingface-hub datasets keras_nlp 
# !pip install -q transformers==4.20.0 keras_nlp==0.3.0 datasets huggingface-hub nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

## **Loading Datasets**

In [32]:
from datasets import load_dataset

dataset_checkpoint = 'xsum'
# raw_datasets = load_dataset(dataset_checkpoint, split="train")
# raw_datasets = load_dataset(dataset_checkpoint)

train_ds = load_dataset(dataset_checkpoint, split='train[:1%]')
test_ds = load_dataset(dataset_checkpoint, split='test[:5%]')
val_ds = load_dataset(dataset_checkpoint, split='validation[:5%]')

In [33]:
train_ds, test_ds, val_ds
# raw_datasets

(Dataset({
     features: ['document', 'summary', 'id'],
     num_rows: 2040
 }),
 Dataset({
     features: ['document', 'summary', 'id'],
     num_rows: 567
 }),
 Dataset({
     features: ['document', 'summary', 'id'],
     num_rows: 567
 }))

In [34]:
# train_split = 0.1
# test_split = 0.02

# split_dataset = raw_datasets.train_test_split(
#     train_size=train_split, test_size=test_split
# )

In [35]:
# split_dataset

## **Preprocessing Data**

In [36]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [37]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [8774, 6, 48, 19, 3, 9, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [38]:
prefix = 'summarize: '
max_input_length = 1024
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(text=inputs, max_length=max_input_length, truncation=True)
    # with tokenizer.as_target_tokenizer():
    #     labels = tokenizer(
    #         examples["summary"], max_length=max_target_length, truncation=True
    #     )
    labels = tokenizer(text_target=examples['summary'], max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [39]:
# split_dataset['train'][1]['summary']
train_ds[:1]

{'document': ['The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreci

In [40]:
preprocess_function(train_ds[:2])

{'input_ids': [[21603, 10, 37, 423, 583, 13, 1783, 16, 20126, 16496, 6, 80, 13, 8, 844, 6025, 4161, 6, 19, 341, 271, 14841, 5, 7057, 161, 19, 4912, 16, 1626, 5981, 11, 186, 7540, 16, 1276, 15, 2296, 7, 5718, 2367, 14621, 4161, 57, 4125, 387, 5, 15059, 7, 30, 8, 4653, 4939, 711, 747, 522, 17879, 788, 12, 1783, 44, 8, 15763, 6029, 1813, 9, 7472, 5, 1404, 1623, 11, 5699, 277, 130, 4161, 57, 18368, 16, 20126, 16496, 227, 8, 2473, 5895, 15, 147, 89, 22411, 139, 8, 1511, 5, 1485, 3271, 3, 21926, 9, 472, 19623, 5251, 8, 616, 12, 15614, 8, 1783, 5, 37, 13818, 10564, 15, 26, 3, 9, 3, 19513, 1481, 6, 18368, 186, 1328, 2605, 30, 7488, 1887, 3, 18, 8, 711, 2309, 9517, 89, 355, 5, 3966, 1954, 9233, 15, 6, 113, 293, 7, 8, 16548, 13363, 106, 14022, 84, 47, 14621, 4161, 6, 243, 255, 228, 59, 7828, 8, 1249, 18, 545, 11298, 1773, 728, 8, 8347, 1560, 5, 611, 6, 255, 243, 72, 1709, 1528, 161, 228, 43, 118, 4006, 91, 12, 766, 8, 3, 19513, 1481, 410, 59, 5124, 5, 96, 196, 17, 19, 1256, 68, 27, 103, 317, 132

In [41]:
train_tokenized_ds = train_ds.map(preprocess_function, batched=True)
test_tokenized_ds = test_ds.map(preprocess_function, batched=True)
val_tokenized_ds = val_ds.map(preprocess_function, batched=True)

# tokenized_dataset = split_dataset.map(preprocess_function, batched=True)

## Fine-Tuned the model

In [42]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [43]:
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [44]:
train_tokenized_ds, test_tokenized_ds, val_tokenized_ds
# tokenized_dataset

(Dataset({
     features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 2040
 }),
 Dataset({
     features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 567
 }),
 Dataset({
     features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 567
 }))

In [45]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="np")

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="np", pad_to_multiple_of=128)

# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [46]:
train_dataset = train_tokenized_ds.to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = test_tokenized_ds.to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
generation_dataset = val_tokenized_ds.to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=generation_data_collator
)
# train_dataset = tokenized_dataset["train"].to_tf_dataset(
#     batch_size=batch_size,
#     columns=["input_ids", "attention_mask", "labels"],
#     shuffle=True,
#     collate_fn=data_collator,
# )
# test_dataset = tokenized_dataset["test"].to_tf_dataset(
#     batch_size=batch_size,
#     columns=["input_ids", "attention_mask", "labels"],
#     shuffle=False,
#     collate_fn=data_collator,
# )
# generation_dataset = (
#     tokenized_dataset["test"]
#     .shuffle()
#     .select(list(range(200)))
#     .to_tf_dataset(
#         batch_size=batch_size,
#         columns=["input_ids", "attention_mask", "labels"],
#         shuffle=False,
#         collate_fn=data_collator,
#     )
# )

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [47]:
type(test_dataset), generation_dataset

(tensorflow.python.data.ops.prefetch_op._PrefetchDataset,
 <_PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'labels': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)}>)

In [48]:
from transformers import AdamWeightDecay
import tensorflow as tf
# from tensorflow import keras

optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
# optimizer = AdamWeightDecay(learning_rate=learning_rate)
# optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [49]:
# !pip install --q rouge-score

In [50]:
# !pip install -q evaluate

In [51]:
import numpy as np
# from datasets import load_metric
from evaluate import load
import nltk
# nltk.download('all')

# metric = load_metric("rouge")
metric = load('rouge')
def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return result

In [52]:
# import keras_nlp
# rouge_l = keras_nlp.metrics.RougeL()

# def metric_fn(eval_predictions):
#     predictions, labels = eval_predictions
#     decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     for label in labels:
#         label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     result = rouge_l(decoded_labels, decoded_predictions)
#     # We will print only the F1 score, you can use other aggregation metrics as well
#     result = {"RougeL": result["f1_score"]}

#     return result

In [53]:
from transformers.keras_callbacks import KerasMetricCallback

# metric_callback = KerasMetricCallback(
#     metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
# )
metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True, use_xla_generation=True
)

callbacks = [metric_callback]

In [54]:
model.fit(
    train_dataset, validation_data=generation_dataset, epochs=1, callbacks=callbacks
)

255/255 [==============================] - ETA: 0s - loss: 3.2059

/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  return py_builtins.overload_of(f)(*args)


255/255 [==============================] - 301s 1s/step - loss: 3.2059 - val_loss: 2.7889 - rouge1: 20.8742 - rouge2: 4.2652 - rougeL: 16.3154 - rougeLsum: 16.3853 - gen_len: 18.5291


## Inference

In [ ]:
document = 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we\'re neglected or forgotten," she said.\n"That may not be true but it is perhaps my perspective over the last few days.\n"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"\nMeanwhile, a flood alert remains in place across the Borders because of the constant rain.\nPeebles was badly hit by problems, sparking calls to introduce more defences in the area.\nScottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.\nThe Labour Party\'s deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.\nHe said it was important to get the flood protection plan right but backed calls to speed up the process.\n"I was quite taken aback by the amount of damage that has been done," he said.\n"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."\nHe said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.\nHave you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.'
document = "summarize: " + document
tokenized = tokenizer([document], return_tensors='np')
out = model.generate(**tokenized, max_length=128)

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0]))

In [ ]:
print('hek')